# Crystal structure of the SARS-CoV-1 RBD bound by the cross-reactive single-domain antibody SARS VHH-72

## https://www.rcsb.org/structure/6WAQ

In [1]:
import nglview as nv
from Bio import PDB
from Bio.SeqUtils import molecular_weight
from Bio.SeqUtils import seq1

In [438]:
view = nv.show_file("6waq.pdb")
view

NGLWidget()

## The pdb file is made up of two christalized copies of vhh72 bound to to Sars-Cov-2 spike protein (chain A bound to chain B and chain C bound to chain D, chain C and D not shown on figure). since our protein of interest is the vhh72 nanobody, we need to extract only this (chain A)

In [194]:
# Class for selecting a specific chain and removing water molecules
class ChainAWithoutWaterSelect(PDB.Select):
    def __init__(self, chain_id):
        self.chain_id = chain_id

    def accept_chain(self, chain):
        return chain.id == self.chain_id

    def accept_residue(self, residue):
        return residue.id[0] == " "  # Accept only standard residues, not waters or heteroatoms

# Load the PDB file
parser = PDB.PDBParser(QUIET=True)
structure = parser.get_structure("structure", "6waq.pdb")
linker = parser.get_structure("linker", "linker.pdb")

# Extract chain A and remove water molecules
chain_id = 'A'
io = PDB.PDBIO()
io.set_structure(structure)
io.save("vhh72.pdb", ChainAWithoutWaterSelect(chain_id))

# Function to calculate properties
def calculate_properties(chain):
    mass = 0.0
    charge = 0
    atom_count = 0
    sequence = ""

    # Iterate through residues and atoms
    for residue in chain:
        if PDB.is_aa(residue, standard=True):
            seq_res = seq1(residue.resname)
            sequence += seq_res
            mass += molecular_weight(seq_res, seq_type="protein")
            for atom in residue:
                atom_count += 1
                # Approximate charge based on residue type
                if seq_res in "KRH":
                    charge += 1  # Basic residues
                elif seq_res in "DE":
                    charge -= 1  # Acidic residues

    return mass, charge, atom_count, sequence

# Load the cleaned structure
#vhh_structure = parser.get_structure("vhh_structure", "vhh72.pdb")

# Get chain A from the cleaned structure
#chain_A = cleaned_structure[0][chain_id]

# Calculate properties
mass, charge, atom_count, sequence = calculate_properties(chain_A)

# Output the results
print(f"Chain ID: {chain_id}")
print(f"Number of atoms: {atom_count}")
print(f"Molecular mass: {mass:.2f} Da")
print(f"Net charge: {charge}")
print(f"Amino acid sequence: {sequence}")
print(f"Sequence : {len(sequence)}")

Chain ID: A
Number of atoms: 970
Molecular mass: 16042.88 Da
Net charge: 1
Amino acid sequence: QVQLQESGGGLVQAGGSLRLSCAASGRTFSEYAMGWFRQAPGKEREFVATISWSGGSTYYTDSVKGRFTISRDNAKNTVYLQMNSLKPDDTAVYYCAAAGLGTVVSEWDYDYDYWGQGTQVTVSSGS
Sequence : 127


In [439]:
view = nv.show_file("vhh72.pdb")
view

NGLWidget()

## Now that we have our Vhh72 extracted without water molecules becauses NAMD will raise errors when generating structural file from files with water molecules.

## Let's introduce our VHH linker below. the VHH linker was designed in Spartan 24 and same result was optain from ChemDraw3D

In [440]:
nv.show_file("linker.pdb")

NGLWidget()

## There is no gold atom connected to the linker because the charm force field algorithm crashes when generating parameter files with when running a designed ligand with metal atom included. we'll manually attached the the gold atom

In [280]:
vhh = parser.get_structure("vhh", "vhh72.pdb")

last_residue = None
for model in vhh:
  for chain in model:
    for residue in chain:
      last_residue = residue  # Update last_residue for each residue

# Access information of the last residue (if found)
if last_residue:
    resname = {}
    for atom in last_residue.get_atoms():
        resname[atom.get_name()] = atom.coord
else:
    print("No residues found in the structure.")


## The connection between the vhh and linker will be made on the carbonyl atom of the serine residue. So, lets get the coordinate of the carbonyl atom only.

In [281]:
ref_atom1 = {}
for atom_name, coordinates in resname.items():
  ref_atom1[atom_name] = coordinates.tolist()
ref_atom1

{'N': [42.88399887084961, 62.0260009765625, 5.833000183105469],
 'CA': [43.6609992980957, 61.09299850463867, 6.5320000648498535],
 'C': [44.49700164794922, 61.584999084472656, 7.7129998207092285],
 'O': [45.71500015258789, 61.63100051879883, 7.5960001945495605],
 'CB': [44.59199905395508, 60.349998474121094, 5.539000034332275],
 'OG': [45.847999572753906, 60.992000579833984, 5.370999813079834]}

## Get the coordinates of the linker to calculate the translocation vector

### In order to avoid errors when iterating over each atom of the linker pdb file, you need to upload the pdb file to charmm-gui ligand modeller to generate charmm force field files and a more appropriate pdb file. https://charmm-gui.org/?doc=input/ligandrm

In [170]:
#download and untar the charmm file
!tar -xf charmm-gui.tgz

In [173]:
!mv charmm-gui-1991583921/ charmm-gui-file

mv: cannot stat 'charmm-gui-1991583921/': No such file or directory


In [322]:
ligandrm = parser.get_structure("ligandrm", "charmm-gui-file/ligandrm.pdb")
# Iterate through models, chains, residues, and atoms
ref_linker_atom = {}
for model in ligandrm:
    for chain in model:
        for residue in chain:
            for atom in residue:
                # Get the current coordinates
                current_coords = atom.get_coord()
                ref_linker_atom[atom.name] = current_coords.tolist()
                print(atom.name, current_coords)

S [1.766 4.617 2.57 ]
C1 [1.769 2.785 2.43 ]
H1 [1.785 2.359 3.458]
H2 [2.7   2.442 1.926]
C2 [0.545 2.19  1.665]
H3 [-0.378  2.503  2.199]
N1 [0.511 2.81  0.274]
H4 [ 1.407  3.035 -0.152]
H5 [-0.159  2.48  -0.413]
C3 [0.607 0.648 1.764]
O [0.613 0.056 2.843]
H6 [ 0.612  0.447 -0.254]
N2 [ 0.669 -0.039  0.611]
C4 [ 0.854 -1.457  0.535]
H7 [ 0.146 -1.981  1.218]
H8 [ 1.886 -1.703  0.874]
C5 [ 0.656 -1.936 -0.837]
H9 [ 0.375 -2.624 -3.004]
C6 [ 0.509 -2.291 -1.993]
H10 [2.92  4.72  3.211]


In [323]:
#Get th reference coordinate of the linker atom
ref_linker_atom["N1"]

[0.5109999775886536, 2.809999942779541, 0.27399998903274536]

## Find the translocation vector

In [324]:
coord_ref_atom_vhh = ref_atom1["C"]
coord_ref_atom_linker = ref_linker_atom["N1"] 

# Calculate the translation vector (difference in coordinates)
translation_vector = [(a1 - a2) for a1, a2 in zip(coord_ref_atom_vhh, coord_ref_atom_linker)]
translation_vector

[43.986001670360565, 58.774999141693115, 7.438999831676483]

## Now apply the translocation vector by moving every atom coordinates by a distance of the translocation vector

In [325]:
# Iterate through each atom of the linker and add the TV
ref_linker_atom = {}
for model in ligandrm:
    for chain in model:
        for residue in chain:
            for atom in residue:
                current_coords = atom.get_coord()
                new_coords = [coord + translation for coord, translation in zip(current_coords, translation_vector)]
                atom.set_coord(new_coords)

#### bonds are automatically created at close distances, so we have to modify the transvector code up there to maintain and an average distance between the N1 and Carbonyl while keeping other atoms at safer distance preventing unwanted bonds

In [327]:
for model in ligandrm:
    for chain in model:
        for residue in chain:
            for atom in residue:
                current_coords = atom.get_coord()
                x, y, z = atom.get_coord()
                atom.set_coord([x - 1, y, z + 1])

In [328]:
#safe the new coordinates
io = PDBIO()
io.set_structure(ligandrm)
io.save("translated_linker.pdb")

#### Create a new pdb file with the combine structure of both nanobody and the new linker moved to the binding site of the nanobody

In [361]:
# Open both input files and the output file
with open("vhh72.pdb", "r") as vhh_file, \
     open("translated_linker.pdb", "r") as translocated_file, \
     open("vhh_linker.pdb", "w") as output_file:

    # write the vhh information to a new file excluding the last line containing the end statement.
    lines_to_copy = []
    for line in vhh_file:
        lines_to_copy.append(line)
    lines_to_copy = lines_to_copy[:-1]
    output_file.writelines(lines_to_copy)
         
    # Copy lines from translocated_linker.pdb to the output file
    for line in translocated_file:
        if line.startswith(('ATOM')):
            current_atom_name = line[12:16].strip()
            current_res_name = line[17:20].strip()
            # Skip the line if it matches the atom name and residue name
            if current_atom_name == 'H4' and current_res_name == 'LIG':
                continue
            if current_atom_name == 'H10' and current_res_name == 'LIG':
                continue
        # Write all other lines to the output file
        output_file.write(line)

print("Combined structure written to output_file")


Combined structure written to output_file


#### In the above process when writing the linker coordinates, we exluced one of the nitrogen's hydrogen. In order to form carbamoylation, Nitrogen looses one hydrogen to bond with the carbonyl atom. We all excluded the sulfur hydrogen since we will be introducing the gold atom below

### Add description to the new file

In [362]:
output_combined_file = "vhh_linker.pdb"
description = "HEADER      CRYSTAL STRUCTURE OF VHH72 CONNECTED TO LINKER\n"
description += "TITLE        SINGLE-DOMAIN ANTIBODY VHH-72\n"
description += "TITLE        2-AMINO-3-MERCAPTO-N-(PROP-2-YNYL)PROPIONAMIDE\n"
description += "REMARK      LINKER FROM PUBCHEM, DISIGNED IN SPARTAN TRANSFORMED IN CHARMM-GUI\n"
description += "REMARK      DATE:     7/ 2/24      6:24:39      CREATED BY USER: VAir\n"
gold_atom = "HETATM   20  AU  LIG L   1      44.752  64.392  11.009  1.00  0.00      LIG AU\n"
connect_au = "CONECT    1   20\n"

# Read the existing contents of vhh_linker.pdb
with open(output_combined_file, "r") as f:
    lines = f.readlines()

# Insert the description at the beginning of the file
lines.insert(0, description)
lines.insert(-2, gold_atom)
lines.insert(-1, connect_au)

# Write the updated contents back to vhh_linker.pdb
with open(output_combined_file, "w") as f:
    f.writelines(lines)

print(f"Description added to {output_combined_file}")

Description added to vhh_linker.pdb


In [415]:
# Load the structure file (assuming vhh_linker.pdb is your structure file)
view = nv.show_structure_file("vhh_linker.pdb")

# Change the representation to CPK (spacefill)
view.clear_representations()
view.add_representation("ball+stick", selection="all")
view

NGLWidget()

#### Spartan designs ligand with gold atom but charmm-gui crashes when transformming a ligand with gold atom, so we would add this gold atom manually, and later in the NAMD force field simulaitons, we will add the gold atom parameters coordinates manually in the charmm force field files

#### Bellow we'll upload the protein linker structure to fpocket to find the binding pocket. this will help to observe the orientation of the structure and weather the linker will in anyway obstruct the activity of the nanobody.

In [416]:
view = view = nv.show_structure_file("vhh_linker_pocket.pdb")

view.clear_representations()
view.add_representation("ball+stick", selection=":A")
view.add_representation("spacefill", selection=":L")
view.add_representation("surface", selection="1:C", color = "red")
view.add_representation("surface", selection="2:C")
view.add_representation("surface", selection="3:C", color = "green")
view.add_representation("surface", selection="4:C", color = "pink")
view.add_representation("surface", selection="5:C", color = "purple")
view

NGLWidget()

#### Fpocket predicted 5 pockets, with pocket 1 (red) being the most prominent with the largets surface volume for van der waals interactions.

In [436]:
view = nv.show_structure_file("vhh_linker_pocket.pdb")

view.clear_representations()
view.add_representation("ball+stick", selection=":A")
view.add_representation("spacefill", selection=":L")
view.add_representation("surface", selection="1:C", color = "red")
view

NGLWidget()

### Now let's proceed to namd to start the molecular dynamics simulations. we will start by solvating the system and predicting any missing atoms should in cases there is one to avoid simulation errors donwlstream. first let's use charmm-gui to generate the force field parameters